# Interpolation

The morphing/iterpolation scheme used by $\mathtt{HistFactory}$.

The treatment of systematic uncertainties is subtle, particularly when one wishes to take into account the correlated effect of multiple sources of systematic uncertainty across many signal and background samples. The most important conceptual issue is that we separate the source of the uncertainty (for instance the uncertainty in the calorimeter’s response to jets) from its effect on an individual signal or background sample (eg. the change in the acceptance and shape of a W +jets background). In particular, the same source of uncertainty has a different effect on the various signal and background samples 1. The effect of these "$\pm1\sigma$" variations about the nominal predictions $\eta_{s}^0$ = 1 and $\sigma^\pm_{spb}$ is quantified by dedicated studies that provide $\eta_{sp}^\pm$ and $\sigma^\pm_{spb}$. The result of these studies can be arranged in tables like those below.

In [15]:
from IPython.display import HTML, display
import tabulate
table = [["Syst","Sample 1", "..." ,"Sample N"],
         ["Nominal Value",r"$\eta^0_{s=1}$=1","...",r"$\eta^0_{s=N}$=1"],
        ["p=$\mathtt{OverallSys}$ 1",r"$\eta^+_{p=1,s=1}$, $\eta^-_{p=1,s=1}$","...",r"$\eta^+_{p=1,s=N}$, $\eta^-_{p=1,s=N}$"],
        [r"$\vdots$",r"$\vdots$",r"$\ddots$",r"$\vdots$"],
        ["p=$\mathtt{OverallSys}$ M",r"$\eta^+_{p=M,s=1}$, $\eta^-_{p=M,s=1}$","...",r"$\eta^+_{p=M,s=N}$, $\eta^-_{p=M,s=N}$"],
        ["Net Effect",r"$\eta_{s=1}(\boldsymbol{\alpha})$","...",r"$\eta_{s=N}(\boldsymbol{\alpha})$"]]
display(HTML(tabulate.tabulate(table, tablefmt='html',headers="firstrow")))
print "and for the histograms"
othertable = [["Syst","Sample 1", "..." ,"Sample N"],
         ["Nominal Value",r"$\sigma^0_{s=1,b}$","...",r"$\sigma^0_{s=N,b}$"],
        ["p=$\mathtt{HisoSys}$ 1",r"$\sigma^+_{p=1,s=1,b}$, $\sigma^-_{p=1,s=1,b}$","...",r"$\sigma^+_{p=1,s=N,b}$, $\sigma^-_{p=1,s=N,b}$"],
        [r"$\vdots$",r"$\vdots$",r"$\ddots$",r"$\vdots$"],
        ["p=$\mathtt{HistoSys}$ M",r"$\sigma^+_{p=M,s=1,b}$, $\sigma^-_{p=M,s=1,b}$","...",r"$\sigma^+_{p=M,s=N,b}$, $\sigma^-_{p=M,s=N,b}$"],
        ["Net Effect",r"$\sigma_{s=1,b}(\boldsymbol{\alpha})$","...",r"$\sigma_{s=N,b}(\boldsymbol{\alpha})$"]]
display(HTML(tabulate.tabulate(othertable, tablefmt='html',headers="firstrow")))

Syst,Sample 1,...,Sample N
Nominal Value,$\eta^0_{s=1}$=1,...,$\eta^0_{s=N}$=1
p=$\mathtt{OverallSys}$ 1,"$\eta^+_{p=1,s=1}$, $\eta^-_{p=1,s=1}$",...,"$\eta^+_{p=1,s=N}$, $\eta^-_{p=1,s=N}$"
$\vdots$,$\vdots$,$\ddots$,$\vdots$
p=$\mathtt{OverallSys}$ M,"$\eta^+_{p=M,s=1}$, $\eta^-_{p=M,s=1}$",...,"$\eta^+_{p=M,s=N}$, $\eta^-_{p=M,s=N}$"
Net Effect,$\eta_{s=1}(\boldsymbol{\alpha})$,...,$\eta_{s=N}(\boldsymbol{\alpha})$


and for the histograms


Syst,Sample 1,...,Sample N
Nominal Value,"$\sigma^0_{s=1,b}$",...,"$\sigma^0_{s=N,b}$"
p=$\mathtt{HisoSys}$ 1,"$\sigma^+_{p=1,s=1,b}$, $\sigma^-_{p=1,s=1,b}$",...,"$\sigma^+_{p=1,s=N,b}$, $\sigma^-_{p=1,s=N,b}$"
$\vdots$,$\vdots$,$\ddots$,$\vdots$
p=$\mathtt{HistoSys}$ M,"$\sigma^+_{p=M,s=1,b}$, $\sigma^-_{p=M,s=1,b}$",...,"$\sigma^+_{p=M,s=N,b}$, $\sigma^-_{p=M,s=N,b}$"
Net Effect,"$\sigma_{s=1,b}(\boldsymbol{\alpha})$",...,"$\sigma_{s=N,b}(\boldsymbol{\alpha})$"


Once one has tabulated the effects of the individual sources of systematic uncertainty as above, one must address two related issues to form a likelihood parametrized with continuous nuisance parameters. First, one must provide an interpolation algorithm to interpolate to define $\eta_{s}(\boldsymbol{\alpha})$ and $\sigma_{sb}(\boldsymbol{\alpha})$. Secondly, one must incorporate constraint terms on the $\alpha_p$ to reflect that the uncertain parameter has been estimated with some uncertainty by an auxiliary mea- surement. A strength of the histogram template based approach (compared to parametrized analytic functions) is that the effect of individual systematics are tracked explicitly; however, the ambiguities associated to the interpolation and constraints are a weakness.

## Interpolation Options

For each sample, one can interpolate and extrapolate from the nominal prediction $\eta_{s}^{0}$ = 1 and the variations $\eta^\pm_{ps}$ to produce a parametrized $\eta_s(\boldsymbol{\alpha})$. Similarly, one can interpolate and extrapolate from the nominal shape $\sigma^0_{sb}$ and the variations $\sigma^\pm_{psb}$ to produce a parametrized $\sigma_{sb}(\boldsymbol{\alpha})$. We choose to parametrize $\alpha_p$ such that $\alpha_p$ = 0 is the nominal value of this parameter, $\alpha_p = \pm 1$ are the “$\pm1\sigma$ variations”. Needless to say, there is a significant amount of ambiguity in these interpolation and extrapolation procedures and they must be handled with care. In the future the $\mathtt{HistFactory}$ may support other types of shape interpolation, but as of ROOT 5.32 the shape interpolation is a ’vertical’ style interpolation that is treated independently per-bin. Four interpolation strategies are described below and can be compared below.

### Piecewise Linear (InterpCode=0)
The piecewise-linear interpolation strategy is defined as
$$
\eta_s(\boldsymbol{\alpha})=1+\sum_{p\in\mathrm{Syst}}I_\mathrm{lin.}(\alpha_p;1,\eta_{sp}^+,\eta_{sp}^-)
$$
and for shape interpolation it is
$$
\sigma_{sb}(\boldsymbol{\alpha})=\sigma^0_{sb}+\sum_{p\in\mathrm{Syst}}I_\mathrm{lin.}(\alpha_p;\eta_{sb}^0,\eta_{psb}^+,\eta_{psb}^-)
$$
with 
$$
I_\mathrm{lin.}(\alpha;I^0,I^+,I^-)=
\begin{cases}
\alpha(I^+-I^0) & \alpha\geq 0 \\
\alpha(I^0-I^-) & \alpha\lt 0 \\
\end{cases}
$$

**Pros**: This approach is the most straightforward of the interpolation strategies.

**Cons**: It has two negative features. First, there is a kink (discontinuous first derivative) at $\alpha$ = 0, which can cause some difficulties for numerical minimization packages such as $\mathtt{Minuit}$. Second, the interpolation factor can extrapolate to negative values. For instance, if $\eta^−$ = 0.5 then we have $\eta(\alpha)$ < 0 when $\alpha$ < −2 (see Fig 1(c)).

Note that this is the default convention for σsb(α) (ie. $\mathtt{HistoSys}$ ).